## Yolo11 tests

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os , sys
from pathlib import Path
cur_path = Path(os.getcwd()).parent.parent
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# from pathlib import Path
# import sys
# import os
# # Add the octron module path to sys.path

# # Get the parent directory of the current working directory
# project_root = Path(os.getcwd()).parent

# # Make sure the octron module is in the path
# if project_root not in sys.path:
#     sys.path.append(str(project_root))

# # You may need to check if the structure is correct
# print(f"Project root: {project_root}")
# print(f"Modules in project root: {[p.name for p in project_root.iterdir() if p.is_dir() and not p.name.startswith('.')]}")

In [ ]:
from octron.yolo_octron.yolo_octron import YOLO_octron
from octron.yolo_octron.helpers.yolo_checks import check_yolo_models
from octron.yolo_octron.helpers.training import collect_labels, draw_polygons

### Organize training data 

In [ ]:
project_path = Path('/Users/horst/Downloads/octo_paralarvae')
yolo_octron = YOLO_octron()

In [ ]:
yolo_octron.project_path = project_path
yolo_octron.prepare_labels(verbose=True, prune_empty_labels=False)
for no_entry, _, label, f_no, _ in yolo_octron.prepare_polygons():
    pass

In [ ]:
for dic in yolo_octron.label_dict.values():
    draw_polygons(dic,
                  dic['video'],
                  max_to_plot=5,
                  )

### Prediction

In [ ]:
from octron.sam2_octron.helpers.video_loader import probe_video
from napari_pyav._reader import FastVideoReader


In [ ]:
vpath = Path('/Users/horst/Downloads/Mouse pupil/Test/Radnitz_2017-01-12_videos_eye_clip.mp4')
video_dict = probe_video(vpath) 
video_reader= FastVideoReader(vpath, read_format='rgb24')

videos_to_predict = {} 
videos_to_predict[vpath] = video_dict
videos_to_predict[vpath]['video'] = video_reader


In [ ]:
model_predict_path = Path('/Users/horst/Downloads/Mouse pupil/model/training/weights/last.pt')
device_label = 'cpu'
yolo_tracker_name = 'bytetrack' # BoTSORT
iou_thresh = .6
conf_thresh = .6
polygon_sigma = 1.
overwrite_predictions = True

In [ ]:
# from tqdm.notebook import tqdm

# # Initialize tqdm progress bar
# total_frames = videos_to_predict[vpath]['num_frames']
# with tqdm(total=total_frames, desc="Processing frames", unit="frames") as pbar:
#     for progress_info in yolo_octron.predict_batch(
#                                         videos_dict=videos_to_predict,
#                                         model_path=model_predict_path,
#                                         device=device_label,
#                                         tracker_name=yolo_tracker_name,
#                                         one_object_per_label=False,
#                                         iou_thresh=iou_thresh,
#                                         conf_thresh=conf_thresh,
#                                         polygon_sigma=polygon_sigma,
#                                         overwrite=overwrite_predictions, 
#                                     ):
#         # Update the progress bar
#         try:
#             pbar.update(progress_info['frame'] - pbar.n)
#         except Exception as e:
#             pass
            